<a href="https://colab.research.google.com/github/fregean/data_analytics_with_python/blob/section2/cleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 データクレンジング

##  データの入手・結合・読込み

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

In [ ]:
URL = 'https://www.meti.go.jp/policy/economy/distribution/daikibo/todokede.html'
res = requests.get(URL)
soup = BeautifulSoup(res.content, 'lxml') # バイト文字列を読み込んで文字コードを推定してくれる # 0.03秒

In [ ]:
DIR = './drive/MyDrive/data/'
 
if not os.path.exists(DIR):
    # ディレクトリが存在しない場合、ディレクトリを作成する
    os.makedirs(DIR)

In [ ]:
for t in soup.select('a'):

  if '法第5条第1項（EXCEL形式' in t.text:
    href = t.get('href')

    if 'daikibo' in href:
      link = 'https://www.meti.go.jp' + href
    else:
      link = 'https://www.meti.go.jp/policy/economy/distribution/daikibo/' + href

    file = requests.get(link)
    filename = os.path.basename(link)

    with open(DIR + filename, 'wb') as f:
      f.write(file.content)

      time.sleep(3)

In [ ]:
!pip install mojimoji
import time
import pandas as pd
import numpy as np

import datetime
import mojimoji
import re
from selenium import webdriver

pd.set_option('display.max_columns', 100)

     |████████████████████████████████| 127 kB 5.1 MB/s 


In [ ]:
dir='drive/MyDrive/data/'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('prefs', {
    'download.default_directory': dir
})

driver = webdriver.Chrome('chromedriver',options=options)
url = 'https://www.meti.go.jp/policy/economy/distribution/daikibo/todokede.html'
driver.get(url)
df_list = pd.read_html(driver.page_source, encoding='cp932')

for num in range(1, 24):

    for value in df_list[num]['EXCEL形式'][df_list[num]['EXCEL形式'].str.contains('法第5条第1項', na=False)]:

        link = driver.find_element_by_link_text(value).click()
        time.sleep(3)


In [ ]:
import glob
 
files = glob.glob(dir + 'ritti_todogai*.xlsx')
files_lst = []

for file in files:

    input_book = pd.ExcelFile(file) 
    input_sheet_name = input_book.sheet_names
    data = input_book.parse(input_sheet_name[0], header=1,
                 skiprows=[0,1,2,3,4,5,6,7], usecols='A:E, G, J:M',
                 names=['局名', '都道府県', '大規模小売店舗名', '所在地', 
                        '小売業者名', '届出日', '開店予定日', '店舗面積', '取下げ'])
    files_lst.append(data)
 
total_data = pd.concat(files_lst)
input_file_name = dir + 'total.xlsx'
total_data.to_excel(input_file_name, index=False)

In [ ]:
dir='drive/MyDrive/data/'
input_file_name = dir + 'total.xlsx'
df = pd.read_excel(input_file_name, sheet_name=0)
df # 12667 rows × 10 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積,取下げ
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1ほか,デンコードー,デンコードー,2019-04-16 00:00:00,2019-12-17 00:00:00,3134,NaN
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13ほか,セントラルリーシングシステム,ジーユー,2019-09-04 00:00:00,2020-05-05 00:00:00,1454,NaN
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番 ほか,リラィアブル,リラィアブル,2019-09-26 00:00:00,2020-05-27 00:00:00,1379,NaN
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４ほか,コープさっぽろ,コープさっぽろ,2019-10-08 00:00:00,2020-06-09 00:00:00,1447,NaN
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23 00:00:00,2020-06-24 00:00:00,1527,NaN
...,...,...,...,...,...,...,...,...,...,...
12662,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04 00:00:00,2022-02-05 00:00:00,1472,NaN
12663,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,Misumi,2021-06-23 00:00:00,2022-02-24 00:00:00,1774,NaN
12664,九州,福岡市,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１外,博多那珂６開発特定目的会社,未定,2021-05-14 00:00:00,2022-04-01 00:00:00,46380,NaN
12665,九州,熊本市,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06 00:00:00,2022-01-04 00:00:00,1670,NaN


In [ ]:
# 大まかにデータフレームの属性を掴む

print(df.shape)
print('---------')
print(df.info())

(12667, 10)
---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12667 entries, 0 to 12666
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   局名        12666 non-null  object
 1   都道府県      12666 non-null  object
 2   大規模小売店舗名  12667 non-null  object
 3   所在地       12665 non-null  object
 4   建物設置者名    12667 non-null  object
 5   小売業者名     12598 non-null  object
 6   届出日       12666 non-null  object
 7   開店予定日     12664 non-null  object
 8   店舗面積      12666 non-null  object
 9   取下げ       275 non-null    object
dtypes: object(10)
memory usage: 989.7+ KB
None


## 異常値・欠損値の処理

In [ ]:
# 「取下げ」列で欠損値でない行のインデックスを抽出
drop_lst = df[df['取下げ'].notna()].index
# 取下げられた届出のインデックスなので、その行は分析に不要（異常値）のため、削除する
df.drop(index=drop_lst, inplace=True) 

df # 12392 rows × 10 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積,取下げ
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1ほか,デンコードー,デンコードー,2019-04-16 00:00:00,2019-12-17 00:00:00,3134,NaN
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13ほか,セントラルリーシングシステム,ジーユー,2019-09-04 00:00:00,2020-05-05 00:00:00,1454,NaN
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番 ほか,リラィアブル,リラィアブル,2019-09-26 00:00:00,2020-05-27 00:00:00,1379,NaN
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４ほか,コープさっぽろ,コープさっぽろ,2019-10-08 00:00:00,2020-06-09 00:00:00,1447,NaN
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23 00:00:00,2020-06-24 00:00:00,1527,NaN
...,...,...,...,...,...,...,...,...,...,...
12662,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04 00:00:00,2022-02-05 00:00:00,1472,NaN
12663,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,Misumi,2021-06-23 00:00:00,2022-02-24 00:00:00,1774,NaN
12664,九州,福岡市,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１外,博多那珂６開発特定目的会社,未定,2021-05-14 00:00:00,2022-04-01 00:00:00,46380,NaN
12665,九州,熊本市,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06 00:00:00,2022-01-04 00:00:00,1670,NaN


In [ ]:
# これで「取下げ」列は欠損値のみ。不要になったので、この列を削除する
df.drop(columns='取下げ', inplace=True)

df # 12392 rows × 9 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1ほか,デンコードー,デンコードー,2019-04-16 00:00:00,2019-12-17 00:00:00,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13ほか,セントラルリーシングシステム,ジーユー,2019-09-04 00:00:00,2020-05-05 00:00:00,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番 ほか,リラィアブル,リラィアブル,2019-09-26 00:00:00,2020-05-27 00:00:00,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４ほか,コープさっぽろ,コープさっぽろ,2019-10-08 00:00:00,2020-06-09 00:00:00,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23 00:00:00,2020-06-24 00:00:00,1527
...,...,...,...,...,...,...,...,...,...
12662,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04 00:00:00,2022-02-05 00:00:00,1472
12663,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,Misumi,2021-06-23 00:00:00,2022-02-24 00:00:00,1774
12664,九州,福岡市,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１外,博多那珂６開発特定目的会社,未定,2021-05-14 00:00:00,2022-04-01 00:00:00,46380
12665,九州,熊本市,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06 00:00:00,2022-01-04 00:00:00,1670


In [ ]:
# それ以外の列で欠損値を含む行を確認する

df[df.isnull().any(axis=1)] # 70 rows × 9 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
236,中部,富山県,ドラッグコスモス新湊店,射水市善光寺19番14 ほか13筆,コスモス薬品,NaN,2019-07-19 00:00:00,2020-03-20 00:00:00,1536
468,東北,岩手県,花巻桜木町複合商業施設,花巻市桜木町二丁目208番地２外,ユニバース,ユニバース,2019-03-08 00:00:00,NaN,3228
692,近畿,滋賀県,ドラッグコスモス大萱店,大津市大萱七丁目字井関3111番ほか,コスモス薬品,NaN,43462,43706,1188
945,北海道,北海道,（仮称）ツルハドラッグ足寄店,足寄郡足寄町南７条１丁目１４，１５,ツルハ,NaN,2021-02-04 00:00:00,2021-02-18 00:00:00,1237
946,北海道,北海道,プロショップホダカ旭川永山店,旭川市永山３条９丁目８１，８２,DCMホーマック,NaN,2021-02-22 00:00:00,2021-10-23 00:00:00,1570
...,...,...,...,...,...,...,...,...,...
12353,関東,茨城県,エスコート赤塚,水戸市赤塚１－３５７－５ 外１７筆,赤塚企画,NaN,2000-10-20 00:00:00,2002-05-18 00:00:00,3641
12392,関東,東京都,ヴェロックス２８,渋谷区神宮前六丁目３１番１７号,ヴェロックス・シティ・インベストメント・リミテッド,NaN,2000-12-05 00:00:00,2001-12-01 00:00:00,2254
12464,近畿,兵庫県,ＡＭＡＧＯＴＴＡ,尼崎市西大物町12番41号,森木材工業,NaN,2000-06-08 00:00:00,2002-10-01 00:00:00,6382
12521,北海道,北海道,ペットワールドPROX音更店,河東郡音更町木野大通東１０丁目１－４ ほか,ルーキーファーム,ジョイフルエーケー,2021-05-19 00:00:00,NaN,1396


In [ ]:
# 推測して埋められる行もあるが、今回は欠損値を含む行は削除する
df.dropna(how='any', inplace=True)

df # 12322 rows × 9 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1ほか,デンコードー,デンコードー,2019-04-16 00:00:00,2019-12-17 00:00:00,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13ほか,セントラルリーシングシステム,ジーユー,2019-09-04 00:00:00,2020-05-05 00:00:00,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番 ほか,リラィアブル,リラィアブル,2019-09-26 00:00:00,2020-05-27 00:00:00,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４ほか,コープさっぽろ,コープさっぽろ,2019-10-08 00:00:00,2020-06-09 00:00:00,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23 00:00:00,2020-06-24 00:00:00,1527
...,...,...,...,...,...,...,...,...,...
12662,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04 00:00:00,2022-02-05 00:00:00,1472
12663,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,Misumi,2021-06-23 00:00:00,2022-02-24 00:00:00,1774
12664,九州,福岡市,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１外,博多那珂６開発特定目的会社,未定,2021-05-14 00:00:00,2022-04-01 00:00:00,46380
12665,九州,熊本市,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06 00:00:00,2022-01-04 00:00:00,1670


In [ ]:
# 欠損値が残っていないかチェックする
df.isnull().sum()

局名          0
都道府県        0
大規模小売店舗名    0
所在地         0
建物設置者名      0
小売業者名       0
届出日         0
開店予定日       0
店舗面積        0
dtype: int64

In [ ]:
# インデックスが所々抜けているのでふり直す

df = df.reset_index(drop=True)

In [ ]:
# 新しいインデックス確認

df # 12322 rows × 9 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1ほか,デンコードー,デンコードー,2019-04-16 00:00:00,2019-12-17 00:00:00,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13ほか,セントラルリーシングシステム,ジーユー,2019-09-04 00:00:00,2020-05-05 00:00:00,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番 ほか,リラィアブル,リラィアブル,2019-09-26 00:00:00,2020-05-27 00:00:00,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４ほか,コープさっぽろ,コープさっぽろ,2019-10-08 00:00:00,2020-06-09 00:00:00,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23 00:00:00,2020-06-24 00:00:00,1527
...,...,...,...,...,...,...,...,...,...
12317,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04 00:00:00,2022-02-05 00:00:00,1472
12318,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,Misumi,2021-06-23 00:00:00,2022-02-24 00:00:00,1774
12319,九州,福岡市,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１外,博多那珂６開発特定目的会社,未定,2021-05-14 00:00:00,2022-04-01 00:00:00,46380
12320,九州,熊本市,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06 00:00:00,2022-01-04 00:00:00,1670


## 日付データの表示変更

pandasは日付をデフォルトで読み込むと文字列型になる

In [ ]:
# データ型を確認する
# 「届出日」「開店予定日」がdatetime型ではない
# https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-types

df.dtypes

局名          object
都道府県        object
大規模小売店舗名    object
所在地         object
建物設置者名      object
小売業者名       object
届出日         object
開店予定日       object
店舗面積        object
dtype: object

In [ ]:
# データ型をそのまま変換しようとするとエラー

#pd.to_datetime(df['届出日'], format='%Y-%m-%d')

### 届出日

エラーの原因は誤った日付の形式が混ざっていることらしい<br>
他に、シリアル値という値も混ざっている（csvシートで確認する）<br>
まずは「届出日」列をシリアル値とそうでないものに分ける

In [ ]:
# シリアル値が含まれるインデックス（フラグ）と数をカウントする関数
from typing import Union

def check_serial(col: pd.Series) -> Union[pd.Series, int]:

    flag = col.astype('str').str.isdigit()

    return flag, flag.sum()

In [ ]:
check_serial.__annotations__

{'col': pandas.core.series.Series,
 'return': typing.Union[pandas.core.series.Series, int]}

In [ ]:
flg, flg_num = check_serial(df['届出日'])
print(f'フラグ：\n{flg}')
print('-----------------')
print(f'フラグの数：{flg_num}')

フラグ：
0        False
1        False
2        False
3        False
4        False
         ...  
12317    False
12318    False
12319    False
12320    False
12321    False
Name: 届出日, Length: 12322, dtype: bool
-----------------
フラグの数：170


In [ ]:
# Length: 170
print(df['届出日'][flg]) # ちなみにシリアル値一つ一つはint型
print('-----------------')
# Length: 12152
print(df['届出日'][~flg]) # ~は否定を表すので、flgが立っていないインデックスを表す

40       43594
82       43655
83       43663
84       43676
85       43684
         ...  
1878     42832
12280    44309
12284    44306
12285    44356
12308    44301
Name: 届出日, Length: 170, dtype: object
-----------------
0        2019-04-16 00:00:00
1        2019-09-04 00:00:00
2        2019-09-26 00:00:00
3        2019-10-08 00:00:00
4        2019-10-23 00:00:00
                ...         
12317    2021-06-04 00:00:00
12318    2021-06-23 00:00:00
12319    2021-05-14 00:00:00
12320    2021-05-06 00:00:00
12321    2021-04-06 00:00:00
Name: 届出日, Length: 12152, dtype: object


In [ ]:
# int型のシリアル値を、datetime型の%Y-%m-%d形式に書き換える

def replace_serial_to_date(col, flag):

    series = pd.to_timedelta(col[flag].astype('float') - 2, unit='D')+pd.to_datetime('1900/1/1')

    return series

In [ ]:
# string型の値を、datetime型の%Y-%m-%d形式に書き換える
# errors='coerce'オプションで、エラーが出る値はNaTに置き換える

def replace_str_to_date(col, flag):

    series = pd.to_datetime(col[flag], format='%Y-%m-%d', errors='coerce')

    return series

In [ ]:
from_serial = replace_serial_to_date(df['届出日'], flg)
from_str = replace_str_to_date(df['届出日'], ~flg)

# Length: 170
print(from_serial) # ちなみにシリアル値一つ一つはint型
print('-----------------')
# Length: 12152
print(from_str) 

40      2019-05-09
82      2019-07-09
83      2019-07-17
84      2019-07-30
85      2019-08-07
           ...    
1878    2017-04-07
12280   2021-04-23
12284   2021-04-20
12285   2021-06-09
12308   2021-04-15
Name: 届出日, Length: 170, dtype: datetime64[ns]
-----------------
0       2019-04-16
1       2019-09-04
2       2019-09-26
3       2019-10-08
4       2019-10-23
           ...    
12317   2021-06-04
12318   2021-06-23
12319   2021-05-14
12320   2021-05-06
12321   2021-04-06
Name: 届出日, Length: 12152, dtype: datetime64[ns]


In [ ]:
# 処理後に欠損値はできてないか確認
print(from_serial[from_serial.isnull()].index)

# 処理中のエラーで欠損値になった値が6つあった
print(from_str[from_str.isnull()].index)

Int64Index([], dtype='int64')
Int64Index([142, 531, 582, 595, 2858, 2861], dtype='int64')


In [ ]:
# 元のデータを確かめると、異常値だった行が欠損値になった様子

df['届出日'][from_str[from_str.isnull()].index]

142        2020月3月5日
531      2,019年3月18日
582       2018 年6月8日
595      2018 年7月27日
2858     2,015年6月11日
2861    2,015年10月13日
Name: 届出日, dtype: object

In [ ]:
# 異常値を修正する値を作成する

d = [datetime.date(2020, 3, 5), datetime.date(2019, 3, 18), datetime.date(2018, 6, 8),
          datetime.date(2018, 7, 27), datetime.date(2015, 6, 11), datetime.date(2015, 10, 13)]

from_str_part = pd.Series(data=d, index=from_str[from_str.isnull()].index, name='届出日')
from_str_part

142     2020-03-05
531     2019-03-18
582     2018-06-08
595     2018-07-27
2858    2015-06-11
2861    2015-10-13
Name: 届出日, dtype: object

In [ ]:
# 2つのSeriesを結合する（from_strの欠損値をfrom_str_partの同じインデックスの値で上書き）
from_str = from_str.combine_first(other=from_str_part)

from_str

0       2019-04-16
1       2019-09-04
2       2019-09-26
3       2019-10-08
4       2019-10-23
           ...    
12317   2021-06-04
12318   2021-06-23
12319   2021-05-14
12320   2021-05-06
12321   2021-04-06
Name: 届出日, Length: 12152, dtype: datetime64[ns]

In [ ]:
# 上書きされていることを確認する
from_str.loc[from_str_part.index]

142    2020-03-05
531    2019-03-18
582    2018-06-08
595    2018-07-27
2858   2015-06-11
2861   2015-10-13
Name: 届出日, dtype: datetime64[ns]

In [ ]:
# from_serialとfrom_strを結合する（両者はインデックスが異なるので上書きは起きない）
# 結合したSeriesで元のDataFrasmeの「届出日」列を上書きする

df['届出日'] = from_serial.combine_first(other=from_str)
df['届出日']

0       2019-04-16
1       2019-09-04
2       2019-09-26
3       2019-10-08
4       2019-10-23
           ...    
12317   2021-06-04
12318   2021-06-23
12319   2021-05-14
12320   2021-05-06
12321   2021-04-06
Name: 届出日, Length: 12322, dtype: datetime64[ns]

### 開店予定日
届出日と同じ処理を行う

In [ ]:
# 「開店予定日」列にシリアル値がいくつあるか数える

flg, flg_num = check_serial(df['開店予定日'])
print(f'フラグ：\n{flg}')
print('-----------------')
print(f'フラグの数：{flg_num}')

フラグ：
0        False
1        False
2        False
3        False
4        False
         ...  
12317    False
12318    False
12319    False
12320    False
12321    False
Name: 開店予定日, Length: 12322, dtype: bool
-----------------
フラグの数：182


In [ ]:
# Length: 182
print(df['開店予定日'][flg]) # ちなみにシリアル値一つ一つはint型
print('-----------------')
# Length: 12140
print(df['開店予定日'][~flg]) # ~は否定を表すので、flgが立っていないインデックスを表す

40       43840
82       43900
83       43908
84       43921
85       43929
         ...  
1791     43220
1792     43191
1824     43090
1878     43077
12308    44546
Name: 開店予定日, Length: 182, dtype: object
-----------------
0        2019-12-17 00:00:00
1        2020-05-05 00:00:00
2        2020-05-27 00:00:00
3        2020-06-09 00:00:00
4        2020-06-24 00:00:00
                ...         
12317    2022-02-05 00:00:00
12318    2022-02-24 00:00:00
12319    2022-04-01 00:00:00
12320    2022-01-04 00:00:00
12321    2021-12-05 00:00:00
Name: 開店予定日, Length: 12140, dtype: object


In [ ]:
# int型のシリアル値をdatetime型へ変換
from_serial = replace_serial_to_date(df['開店予定日'], flg)
# str型をdatetime型へ変換
from_str = replace_str_to_date(df['開店予定日'], ~flg)

# Length: 182
print(from_serial) # ちなみにシリアル値一つ一つはint型
print('-----------------')
# Length: 12140
print(from_str) 

40      2020-01-10
82      2020-03-10
83      2020-03-18
84      2020-03-31
85      2020-04-08
           ...    
1791    2018-04-30
1792    2018-04-01
1824    2017-12-21
1878    2017-12-08
12308   2021-12-16
Name: 開店予定日, Length: 182, dtype: datetime64[ns]
-----------------
0       2019-12-17
1       2020-05-05
2       2020-05-27
3       2020-06-09
4       2020-06-24
           ...    
12317   2022-02-05
12318   2022-02-24
12319   2022-04-01
12320   2022-01-04
12321   2021-12-05
Name: 開店予定日, Length: 12140, dtype: datetime64[ns]


In [ ]:
# 処理後に欠損値はできてないか確認
print(from_serial[from_serial.isnull()].index)

# 処理中のエラーで欠損値になった値が30個あった
print(from_str[from_str.isnull()].index)

Int64Index([], dtype='int64')
Int64Index([  748,   749,   921,  1035,  1228,  1229,  1438,  1474,  2865,
             2999,  4076,  4452,  4782,  4783,  4876,  5312,  5427,  5429,
             5488,  6096,  6613,  6614,  6615,  6716,  7191,  9284, 11704,
            11705, 11800, 12256],
           dtype='int64')


In [ ]:
# 元のデータを確かめると、異常値だった行が欠損値になった様子

df['開店予定日'][from_str[from_str.isnull()].index]

748         2019年12月頃
749        2019年12月1日
921         2019年9月上旬
1035     21021年11月13日
1228           2024年夏
1229           2024年夏
1438       2018年7月30年
1474       2017年12月上旬
2865      2016年11月19日
2999        2016年8月中旬
4076        2014年3月下旬
4452          2013年5月
4782       2012年11月中旬
4783        2013年3月中旬
4876          2013年4月
5312          2012年2月
5427       2012年11月上旬
5429       2012年11月上旬
5488             H25春
6096         2011年4月頃
6613          2011年3月
6614         平成24年度下期
6615         平成24年度下期
6716         2009年10月
7191          平成２３年春頃
9284           2011年春
11704     2003年03月01日
11705     2003年04月01日
11800     2002年11月21日
12256     2021年 12月3日
Name: 開店予定日, dtype: object

In [ ]:
# 異常値を修正する値を作成する

d = [datetime.date(2021, 12, 3), datetime.date(2021, 11, 13), datetime.date(2024, 8, 1),
          datetime.date(2024, 8, 1), datetime.date(2019, 12, 1), datetime.date(2019, 12, 1),
          datetime.date(2019, 9, 1), datetime.date(2018, 7, 30), datetime.date(2017, 12, 1), 
          datetime.date(2016, 11, 19), datetime.date(2016, 8, 15), datetime.date(2014, 3, 30), 
          datetime.date(2013, 5, 1), datetime.date(2012, 11, 15), datetime.date(2013, 3, 15), 
          datetime.date(2013, 4, 1), datetime.date(2012, 2, 1), datetime.date(2012, 11, 1), 
          datetime.date(2012, 11, 1), datetime.date(2013, 4, 1), datetime.date(2011, 4, 1), 
          datetime.date(2011, 3, 1), datetime.date(2012, 10, 1), datetime.date(2012, 10, 1), 
          datetime.date(2009, 10, 1), datetime.date(2011, 4, 1), datetime.date(2011, 4, 1), 
          datetime.date(2003, 3, 1), datetime.date(2003, 4, 1), datetime.date(2002, 11, 21)
          ]

from_str_part = pd.Series(data=d, index=from_str[from_str.isnull()].index, name='開店予定日')
from_str_part

748      2021-12-03
749      2021-11-13
921      2024-08-01
1035     2024-08-01
1228     2019-12-01
1229     2019-12-01
1438     2019-09-01
1474     2018-07-30
2865     2017-12-01
2999     2016-11-19
4076     2016-08-15
4452     2014-03-30
4782     2013-05-01
4783     2012-11-15
4876     2013-03-15
5312     2013-04-01
5427     2012-02-01
5429     2012-11-01
5488     2012-11-01
6096     2013-04-01
6613     2011-04-01
6614     2011-03-01
6615     2012-10-01
6716     2012-10-01
7191     2009-10-01
9284     2011-04-01
11704    2011-04-01
11705    2003-03-01
11800    2003-04-01
12256    2002-11-21
Name: 開店予定日, dtype: object

In [ ]:
# 2つのSeriesを結合する（from_strの欠損値をfrom_str_partの同じインデックスの値で上書き）
from_str = from_str.combine_first(other=from_str_part)

from_str

0       2019-12-17
1       2020-05-05
2       2020-05-27
3       2020-06-09
4       2020-06-24
           ...    
12317   2022-02-05
12318   2022-02-24
12319   2022-04-01
12320   2022-01-04
12321   2021-12-05
Name: 開店予定日, Length: 12140, dtype: datetime64[ns]

In [ ]:

# 上書きされていることを確認する
from_str.loc[from_str_part.index]

748     2021-12-03
749     2021-11-13
921     2024-08-01
1035    2024-08-01
1228    2019-12-01
1229    2019-12-01
1438    2019-09-01
1474    2018-07-30
2865    2017-12-01
2999    2016-11-19
4076    2016-08-15
4452    2014-03-30
4782    2013-05-01
4783    2012-11-15
4876    2013-03-15
5312    2013-04-01
5427    2012-02-01
5429    2012-11-01
5488    2012-11-01
6096    2013-04-01
6613    2011-04-01
6614    2011-03-01
6615    2012-10-01
6716    2012-10-01
7191    2009-10-01
9284    2011-04-01
11704   2011-04-01
11705   2003-03-01
11800   2003-04-01
12256   2002-11-21
Name: 開店予定日, dtype: datetime64[ns]

In [ ]:

# from_serialとfrom_strを結合する（両者はインデックスが異なるので上書きは起きない）
# 結合したSeriesで元のDataFrasmeの「開店予定日」列を上書きする

df['開店予定日'] = from_serial.combine_first(other=from_str)
df['開店予定日']

0       2019-12-17
1       2020-05-05
2       2020-05-27
3       2020-06-09
4       2020-06-24
           ...    
12317   2022-02-05
12318   2022-02-24
12319   2022-04-01
12320   2022-01-04
12321   2021-12-05
Name: 開店予定日, Length: 12322, dtype: datetime64[ns]

In [ ]:
df

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1ほか,デンコードー,デンコードー,2019-04-16,2019-12-17,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13ほか,セントラルリーシングシステム,ジーユー,2019-09-04,2020-05-05,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番 ほか,リラィアブル,リラィアブル,2019-09-26,2020-05-27,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４ほか,コープさっぽろ,コープさっぽろ,2019-10-08,2020-06-09,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23,2020-06-24,1527
...,...,...,...,...,...,...,...,...,...
12317,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04,2022-02-05,1472
12318,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,Misumi,2021-06-23,2022-02-24,1774
12319,九州,福岡市,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１外,博多那珂６開発特定目的会社,未定,2021-05-14,2022-04-01,46380
12320,九州,熊本市,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06,2022-01-04,1670


## 不要な表記の置換え

### 所在地の表記を整形する

In [ ]:
import logging

def clean_address(col):

    places = []
    for idx, place in enumerate(col):
        
        try:

            if '外' in place:
                place = place[:place.rfind('外')]
            elif 'ほか' in place:
                place = place[:place.rfind('ほか')]
            elif '他' in place:
                place = place[:place.rfind('他')]   

            place = ''.join(place.split())

        except:

            place = np.nan
            logging.exception('適切な住所ではありません')
            print(f'インデックス：{idx}')

        places.append(place)
      
    return places

In [ ]:
# 住所の不要な表記を削除する関数実行（返り値はなし）
# エラーになったらそこはNaNで埋める
places = clean_address(df['所在地'])

ERROR:root:適切な住所ではありません
Traceback (most recent call last):
  File "<ipython-input-38-fcc0b2574579>", line 10, in clean_address
    if '外' in place:
TypeError: argument of type 'int' is not iterable


インデックス：8311


In [ ]:
# リストをインデックス付きのSeriesに変換する
places = pd.Series(data=places, index=df.index, name='所在地')
# NaNのあるインデックス8459を確認
print(places[places.isnull()])

8311    NaN
Name: 所在地, dtype: object


In [ ]:
# Seriesを上書きする
df['所在地'] = places

In [ ]:
df[df['所在地'].isnull()]

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
8311,関東,千葉県,（仮称）ユニクロ新鎌ヶ谷店,NaN,澁谷三郎,ユニクロ,2006-05-08,2007-03-01,1302


In [ ]:
# この行を削除する
df.dropna(how='any', inplace=True)

In [ ]:
# インデックス振り直し
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1,デンコードー,デンコードー,2019-04-16,2019-12-17,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13,セントラルリーシングシステム,ジーユー,2019-09-04,2020-05-05,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番,リラィアブル,リラィアブル,2019-09-26,2020-05-27,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４,コープさっぽろ,コープさっぽろ,2019-10-08,2020-06-09,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23,2020-06-24,1527
...,...,...,...,...,...,...,...,...,...
12316,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04,2022-02-05,1472
12317,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,Misumi,2021-06-23,2022-02-24,1774
12318,九州,福岡市,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１,博多那珂６開発特定目的会社,未定,2021-05-14,2022-04-01,46380
12319,九州,熊本市,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06,2022-01-04,1670


In [ ]:
dir = 'drive/MyDrive/data/'

# 不要な表記削除処理まで済んだデータをsession2_7.csvとして保存しておく
# インデックスは出力しない
df.to_csv(dir+'session2_7.csv', index=False)

### 緯度軽度を取得する

https://qiita.com/paulxll/items/7bc4a5b0529a8d784673

## 表記揺れの修正


### 小売業者名の表記の統一

In [ ]:
len(df['小売業者名'].unique())

1923

In [ ]:
# 株式会社の表示を削除する

def clean_labels(data, replace_at=''):
        
    data = re.sub('株式会社|㈱|（株）|\(|(株)|\)|ほか|他', replace_at, data)

    data = ''.join(data.split())

    return data

In [ ]:
# 横棒、中黒記号の類似を統一

def replace_symbol(data, replace_line='ー', replace_point='・'):
    
    data = re.sub('－|-|‐|−|‒|—|–|―|ｰ|─|━|ㅡ|ـ|⁻|₋', replace_line, data)
    data = re.sub('･|・|・|・|•', replace_point, data)

    return data

In [ ]:
# 表記ゆれ

def fix_spelling(col):

    retailer = []
    for d in col:
        
        d = clean_labels(d, replace_at='')
        d = replace_symbol(d, replace_line='ー', replace_point='・')
        d = d.replace('“', '') # ダブルコロン削除
        d = d.upper() # 大文字に統一
        d = mojimoji.han_to_zen(d) # 半角に統一
                
        if 'セブン' in d:
            d = re.sub(r'セブンイレブン', 'セブンーイレブン', d)
            d = re.sub('・ジャパン|ジャパン', '', d)
            
        if 'ルシア' in d:
            d = re.sub(r'ウェルシア', 'ウエルシア', d)
            d = d.replace('薬局', '')
            
        if 'アイ・ティー' in d:
            d = re.sub(r'アイ・ティーエックス', 'アイ・ティー・エックス', d)
            
        # 業者名が連名になっているところは改行する
        if '、' in d:
            d = d.replace('、', '・\n')       

        retailer.append(d)
        
    return retailer

In [ ]:
retailer = fix_spelling(df['小売業者名'])

In [ ]:
# リストをインデックス付きのSeriesに変換する
retailer = pd.Series(data=retailer, index=df.index, name='小売業者名')
# このSeriesで「小売業者名」を上書きする
df['小売業者名'] = retailer

In [ ]:
pd.DataFrame(df['小売業者名'].value_counts())

,小売業者名
コスモス薬品,1008
ヤマダ電機,397
しまむら,367
未定,286
ニトリ,253
...,...
文苑堂,1
ベルナ,1
ダイサダ,1
シバタ,1


### 都道府県名の表記の統一

In [ ]:
# 都道府県名に市の名前が混ざっている

print(df['都道府県'][df['都道府県'].str.contains('市')])
print('----------------')
print(df['都道府県'][df['都道府県'].str.contains('市')].unique())
print(len(df['都道府県'][df['都道府県'].str.contains('市')].unique()))

13       札幌市
14       札幌市
15       札幌市
16       札幌市
17       札幌市
        ... 
12289    岡山市
12290    広島市
12291    岡山市
12318    福岡市
12319    熊本市
Name: 都道府県, Length: 2085, dtype: object
----------------
['札幌市' '仙台市' 'さいたま市' '千葉市' '横浜市' '川崎市' '新潟市' '静岡市' '浜松市' '名古屋市' '京都市'
 '大阪市' '神戸市' '堺市' '広島市' '福岡市' '北九州市' '熊本市' '相模原市' '岡山市']
20


In [ ]:
# フラグにする
city_flg = df['都道府県'].str.contains('市')
city_flg

0        False
1        False
2        False
3        False
4        False
         ...  
12316    False
12317    False
12318     True
12319     True
12320    False
Name: 都道府県, Length: 12321, dtype: bool

In [ ]:
# 上記を除くと、「県」が抜けている表記が3つある
#  '富山', '石川', '沖縄'

df['都道府県'][~city_flg].unique()

array(['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県',
       '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', '山梨県', '長野県', '静岡県',
       '愛知県', '岐阜県', '三重県', '富山県', '石川県', '福井県', '滋賀県', '京都府', '大阪府',
       '兵庫県', '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県',
       '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', '熊本県', '宮崎県', '鹿児島県',
       '沖縄県', '大分県', '沖縄', '富山', '石川'], dtype=object)

### 都道府県名と市町村名の対応

In [ ]:
import os
import pprint
import time
import urllib.error
import urllib.request

def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)

def download_file_to_dir(url, dst_dir):
    download_file(url, os.path.join(dst_dir, os.path.basename(url)))

In [ ]:
# ダウンロードurlからzipファイルを取得する
"""
住所データcsv
http://jusyo.jp/csv/new.php
"""

url = 'http://jusyo.jp/downloads/new/csv/csv_zenkoku.zip'
download_file_to_dir(url, dir)

In [ ]:
import zipfile
import glob
 
files = glob.glob(dir + '*.zip')

with zipfile.ZipFile(files[0]) as existing_zip:
    existing_zip.extractall(dir)

In [ ]:
zenkoku = pd.read_csv(dir + 'zenkoku.csv', encoding='cp932')
zenkoku

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,住所CD,都道府県CD,市区町村CD,町域CD,郵便番号,事業所フラグ,廃止フラグ,都道府県,都道府県カナ,市区町村,市区町村カナ,町域,町域カナ,町域補足,京都通り名,字丁目,字丁目カナ,補足,事業所名,事業所名カナ,事業所住所,新住所CD
0,60000000,1,1101,11010000,060-0000,0,0,北海道,ホッカイドウ,札幌市中央区,サッポロシチュウオウク,NaN,,（該当なし）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60872100,1,1101,11010000,060-8721,1,0,北海道,ホッカイドウ,札幌市中央区,サッポロシチュウオウク,NaN,,（該当なし）,NaN,NaN,NaN,NaN,大丸 株式会社 紙包材営業本部,ダイマル カブシキガイシヤ カミホウザイエイギヨウホンブ,北３条西１４丁目２番,NaN
2,60860200,1,1101,11010000,060-8602,1,0,北海道,ホッカイドウ,札幌市中央区,サッポロシチュウオウク,NaN,,（該当なし）,NaN,NaN,NaN,NaN,株式会社 朝日新聞社 北海道支社,カブシキカイシヤ アサヒシンブンシヤ ホツカイドウシシヤ,北１条西１丁目６番地さっぽろ創生スクエア９階,NaN
3,60855200,1,1101,11010000,060-8552,1,0,北海道,ホッカイドウ,札幌市中央区,サッポロシチュウオウク,NaN,,（該当なし）,NaN,NaN,NaN,NaN,損害保険ジャパン日本興亜 株式会社,ソンガイホケンジヤパンニツポンコウア カブシキガイシヤ,北１条西６丁目２番地損保ジャパン日本興亜札幌ビル,NaN
4,60850400,1,1101,11010000,060-8504,1,0,北海道,ホッカイドウ,札幌市中央区,サッポロシチュウオウク,NaN,,（該当なし）,NaN,NaN,NaN,NaN,富士通 株式会社 北海道支社,フジツウ カブシキガイシヤ ホツカイドウシシヤ,北２条西４丁目１番地札幌三井ＪＰビルディング,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149865,907175100,47,47381,473810012,907-1751,0,0,沖縄県,オキナワケン,八重山郡竹富町,ヤエヤマグンタケトミチョウ,波照間,ハテルマ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149866,907154400,47,47381,473810013,907-1544,0,0,沖縄県,オキナワケン,八重山郡竹富町,ヤエヤマグンタケトミチョウ,鳩間,ハトマ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149867,907180000,47,47382,473820000,907-1800,0,0,沖縄県,オキナワケン,八重山郡与那国町,ヤエヤマグンヨナグニチョウ,NaN,,（該当なし）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149868,907180100,47,47382,473820001,907-1801,0,0,沖縄県,オキナワケン,八重山郡与那国町,ヤエヤマグンヨナグニチョウ,与那国,ヨナグニ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def predict_logic(data, city):     

    hit_area = data[data['市区町村'].str.contains(city)]

    city_candidates = hit_area['都道府県'].unique()
            
    return city_candidates

In [ ]:
# お試し
predict_logic(zenkoku, '北九州市')[0]

'福岡県'

In [ ]:
# 都市と都道府県の変換辞書
cities = df['都道府県'][df['都道府県'].str.contains('市')].unique()

# 広島市は北海道と広島県の候補がある
city_dic = {city : predict_logic(zenkoku, city) for city in cities}
city_dic

{'さいたま市': array(['埼玉県'], dtype=object),
 '京都市': array(['京都府'], dtype=object),
 '仙台市': array(['宮城県'], dtype=object),
 '北九州市': array(['福岡県'], dtype=object),
 '千葉市': array(['千葉県'], dtype=object),
 '名古屋市': array(['愛知県'], dtype=object),
 '堺市': array(['大阪府'], dtype=object),
 '大阪市': array(['大阪府'], dtype=object),
 '岡山市': array(['岡山県'], dtype=object),
 '川崎市': array(['神奈川県'], dtype=object),
 '広島市': array(['北海道', '広島県'], dtype=object),
 '新潟市': array(['新潟県'], dtype=object),
 '札幌市': array(['北海道'], dtype=object),
 '横浜市': array(['神奈川県'], dtype=object),
 '浜松市': array(['静岡県'], dtype=object),
 '熊本市': array(['熊本県'], dtype=object),
 '相模原市': array(['神奈川県'], dtype=object),
 '神戸市': array(['兵庫県'], dtype=object),
 '福岡市': array(['福岡県'], dtype=object),
 '静岡市': array(['静岡県'], dtype=object)}

In [ ]:
# 元のDataFrameで広島市を確認する
# 中国地方の広島市のみ
df['局名'][df['都道府県']=='広島市'].unique()

array(['中国'], dtype=object)

In [ ]:
# 上書きする
city_dic['広島市'] = np.array(['広島県'], dtype=object)
city_dic

{'さいたま市': array(['埼玉県'], dtype=object),
 '京都市': array(['京都府'], dtype=object),
 '仙台市': array(['宮城県'], dtype=object),
 '北九州市': array(['福岡県'], dtype=object),
 '千葉市': array(['千葉県'], dtype=object),
 '名古屋市': array(['愛知県'], dtype=object),
 '堺市': array(['大阪府'], dtype=object),
 '大阪市': array(['大阪府'], dtype=object),
 '岡山市': array(['岡山県'], dtype=object),
 '川崎市': array(['神奈川県'], dtype=object),
 '広島市': array(['広島県'], dtype=object),
 '新潟市': array(['新潟県'], dtype=object),
 '札幌市': array(['北海道'], dtype=object),
 '横浜市': array(['神奈川県'], dtype=object),
 '浜松市': array(['静岡県'], dtype=object),
 '熊本市': array(['熊本県'], dtype=object),
 '相模原市': array(['神奈川県'], dtype=object),
 '神戸市': array(['兵庫県'], dtype=object),
 '福岡市': array(['福岡県'], dtype=object),
 '静岡市': array(['静岡県'], dtype=object)}

In [ ]:
# 辞書のvalueについていたarrayを外す
city_dic = {k: v[0] for k, v in city_dic.items()}
city_dic

{'さいたま市': '埼玉県',
 '京都市': '京都府',
 '仙台市': '宮城県',
 '北九州市': '福岡県',
 '千葉市': '千葉県',
 '名古屋市': '愛知県',
 '堺市': '大阪府',
 '大阪市': '大阪府',
 '岡山市': '岡山県',
 '川崎市': '神奈川県',
 '広島市': '広島県',
 '新潟市': '新潟県',
 '札幌市': '北海道',
 '横浜市': '神奈川県',
 '浜松市': '静岡県',
 '熊本市': '熊本県',
 '相模原市': '神奈川県',
 '神戸市': '兵庫県',
 '福岡市': '福岡県',
 '静岡市': '静岡県'}

In [ ]:
# 置換辞書を用いて一括で置換する
df['都道府県'].replace(city_dic, inplace=True)
df['都道府県'][city_flg]

13       北海道
14       北海道
15       北海道
16       北海道
17       北海道
        ... 
12289    岡山県
12290    広島県
12291    岡山県
12318    福岡県
12319    熊本県
Name: 都道府県, Length: 2085, dtype: object

In [ ]:
# 47都道府県のリストを作成
local = [ v for v in df['都道府県'][~city_flg].unique() if len(v)>2 ]
print(local)
print(len(local))

['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', '山梨県', '長野県', '静岡県', '愛知県', '岐阜県', '三重県', '富山県', '石川県', '福井県', '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', '熊本県', '宮崎県', '鹿児島県', '沖縄県', '大分県']
47


In [ ]:
print(df['都道府県'][~df['都道府県'].isin(local)])
temp = df['都道府県'][~df['都道府県'].isin(local)].unique()
print(temp)

3674     沖縄
3675     沖縄
3676     沖縄
3677     沖縄
3678     沖縄
3679     沖縄
3680     沖縄
4372     沖縄
4373     沖縄
4374     沖縄
5088     沖縄
5089     沖縄
5090     沖縄
5091     沖縄
5092     沖縄
5093     沖縄
5094     沖縄
5095     沖縄
5096     沖縄
5097     沖縄
12255    富山
12256    富山
12257    富山
12258    石川
12259    石川
Name: 都道府県, dtype: object
['沖縄' '富山' '石川']


In [ ]:
# 置換辞書を作成する
temp_dic = {t: t+'県' for t in temp}
temp_dic

{'富山': '富山県', '沖縄': '沖縄県', '石川': '石川県'}

In [ ]:
# 置換辞書で上書きする
df['都道府県'].replace(temp_dic, inplace=True)

### 局名の表記揺れの修正

In [ ]:
df['局名'].unique()

array(['北海道', '東北', '関東', '中部', '近畿', '中国', '四国', '九州', '沖縄', '近畿 ', '*',
       '近畿\u3000', ' 近畿', '栃木県'], dtype=object)

In [ ]:
block = fix_spelling(df['局名'])

In [ ]:
df.isnull().sum()

局名          0
都道府県        0
大規模小売店舗名    0
所在地         0
建物設置者名      0
小売業者名       0
届出日         0
開店予定日       0
店舗面積        0
dtype: int64

In [ ]:
# リストをインデックス付きのSeriesに変換する
block = pd.Series(data=block, index=df.index, name='局名')
# このSeriesで「小売業者名」を上書きする
df['局名'] = block

In [ ]:
# 九地方区分名以外の表記が混ざっている
print(df['局名'].unique())
print('-----------------')
print(df['局名'].unique()[-2:])

['北海道' '東北' '関東' '中部' '近畿' '中国' '四国' '九州' '沖縄' '＊' '栃木県']
-----------------
['＊' '栃木県']


In [ ]:
# 誤った局名の表記をerr_変数に格納する
err_1 = df['局名'].unique()[-1] 
err_2 = df['局名'].unique()[-2] 

# 局名をerr_変数で指定した行を確認する
df[(df['局名']==err_1) | (df['局名']==err_2)]

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
4056,＊,愛知県,大名古屋ビルヂング,名古屋市中村区名駅三丁目2701番,三菱地所,未定,2013-11-05,2015-11-01,7000
10365,栃木県,栃木県,カワチ薬品上三川インター店\n仮称バースディインターパーク店,宇都宮市インターパーク六丁目五番地３,カワチ薬品,カワチ薬品,2003-05-26,2004-01-27,3327


In [ ]:
# 誤った局名の表記に対応する都道府県名を抽出する
print(df['都道府県'][df['局名']==err_1])
print(df['都道府県'][df['局名']==err_2])

# 誤った局名の表記に対応する都道府県名をlocal_変数に格納する
local_1 = df['都道府県'][df['局名']==err_1]
local_2 = df['都道府県'][df['局名']==err_2]

10365    栃木県
Name: 都道府県, dtype: object
4056    愛知県
Name: 都道府県, dtype: object


In [ ]:
# これらの都道府県名に対応する一番多い局名を抽出する
print(df['局名'][df['都道府県']== local_1.values[0]].value_counts().index[0])
print(df['局名'][df['都道府県']== local_2.values[0]].value_counts().index[0])

# これらの都道府県名に対応する一番多い（正しい）局名をblock_変数に格納する
block_1 = df['局名'][df['都道府県']== local_1.values[0]].value_counts().index[0]
block_2 = df['局名'][df['都道府県']== local_2.values[0]].value_counts().index[0]

関東
中部


In [ ]:
# 誤った局名を正しい局名に変換する置換辞書を作成する
block_dic = {err_1 : block_1, err_2 : block_2}

# 置換辞書で置き換えを実行
df['局名'].replace(block_dic, inplace=True)

In [ ]:
# 局名が九地方区分名のみになっている
df['局名'].unique()

array(['北海道', '東北', '関東', '中部', '近畿', '中国', '四国', '九州', '沖縄'],
      dtype=object)

In [ ]:
# データ型の確認
df.dtypes

局名                  object
都道府県                object
大規模小売店舗名            object
所在地                 object
建物設置者名              object
小売業者名               object
届出日         datetime64[ns]
開店予定日       datetime64[ns]
店舗面積                object
dtype: object

In [ ]:
# 欠損値なし
df.isnull().sum()

局名          0
都道府県        0
大規模小売店舗名    0
所在地         0
建物設置者名      0
小売業者名       0
届出日         0
開店予定日       0
店舗面積        0
dtype: int64

In [ ]:
dir = 'drive/MyDrive/data/'

# 欠損値処理まで済んだデータをsection_2.csvとして保存しておく
# インデックスは出力しない
df.to_csv(dir+'section_2.csv', index=False)

In [ ]:
dir = 'drive/MyDrive/data/'

df = pd.read_csv(dir + 'section_2.csv', parse_dates=['届出日', '開店予定日'])
df # 12321 rows × 9 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1,デンコードー,デンコードー,2019-04-16,2019-12-17,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13,セントラルリーシングシステム,ジーユー,2019-09-04,2020-05-05,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番,リラィアブル,リラィアブル,2019-09-26,2020-05-27,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４,コープさっぽろ,コープさっぽろ,2019-10-08,2020-06-09,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23,2020-06-24,1527
...,...,...,...,...,...,...,...,...,...
12316,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04,2022-02-05,1472
12317,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,ＭＩＳＵＭＩ,2021-06-23,2022-02-24,1774
12318,九州,福岡県,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１,博多那珂６開発特定目的会社,未定,2021-05-14,2022-04-01,46380
12319,九州,熊本県,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06,2022-01-04,1670


In [ ]:
df.dtypes

局名                  object
都道府県                object
大規模小売店舗名            object
所在地                 object
建物設置者名              object
小売業者名               object
届出日         datetime64[ns]
開店予定日       datetime64[ns]
店舗面積                object
dtype: object

In [ ]:
# なぜか欠損値が一つある
df.isnull().sum()

局名          0
都道府県        0
大規模小売店舗名    0
所在地         0
建物設置者名      0
小売業者名       1
届出日         0
開店予定日       0
店舗面積        0
dtype: int64

In [ ]:
df[df['小売業者名'].isnull()]

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
10777,近畿,兵庫県,(仮)神戸三宮一丁目ビル,中央区三宮町１－４－５１,安田不動産,NaN,2003-12-25,2004-09-17,5511


In [ ]:
# 欠損値を含む行を削除
df.dropna(how='any', inplace=True)
df # 12320 rows × 9 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1,デンコードー,デンコードー,2019-04-16,2019-12-17,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13,セントラルリーシングシステム,ジーユー,2019-09-04,2020-05-05,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番,リラィアブル,リラィアブル,2019-09-26,2020-05-27,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４,コープさっぽろ,コープさっぽろ,2019-10-08,2020-06-09,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23,2020-06-24,1527
...,...,...,...,...,...,...,...,...,...
12316,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04,2022-02-05,1472
12317,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,ＭＩＳＵＭＩ,2021-06-23,2022-02-24,1774
12318,九州,福岡県,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１,博多那珂６開発特定目的会社,未定,2021-05-14,2022-04-01,46380
12319,九州,熊本県,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06,2022-01-04,1670


In [ ]:
dir = 'drive/MyDrive/data/'

# 欠損値処理まで済んだデータをsection_2.csvとして保存し直し
# インデックスは出力しない
df.to_csv(dir+'section_2.csv', index=False)

In [ ]:
dir = 'drive/MyDrive/data/'

df = pd.read_csv(dir + 'section_2.csv', parse_dates=['届出日', '開店予定日'])
df # 12320 rows × 9 columns

,局名,都道府県,大規模小売店舗名,所在地,建物設置者名,小売業者名,届出日,開店予定日,店舗面積
0,北海道,北海道,木野タウン ケーズデンキ棟,音更町木野大通東11丁目1-1,デンコードー,デンコードー,2019-04-16,2019-12-17,3134
1,北海道,北海道,旭川豊岡タウンプラザ GU棟 SHOE・PLAZA棟,旭川市豊岡4条4丁目1-13,セントラルリーシングシステム,ジーユー,2019-09-04,2020-05-05,1454
2,北海道,北海道,リラィアブルブックス 運動公園通り店,釧路町桂木3丁目7番1、７番2、8番,リラィアブル,リラィアブル,2019-09-26,2020-05-27,1379
3,北海道,北海道,コープさっぽろ とくら店,函館市戸倉町224番－４,コープさっぽろ,コープさっぽろ,2019-10-08,2020-06-09,1447
4,北海道,北海道,ツルハドラッグ旭川永山３条西店,旭川市永山３条８丁目76番１,北海道リース,ツルハ,2019-10-23,2020-06-24,1527
...,...,...,...,...,...,...,...,...,...
12315,九州,鹿児島県,（仮称）ドラッグコスモス鹿屋運動公園前店,鹿屋市西原２丁目104番１,コスモス薬品,コスモス薬品,2021-06-04,2022-02-05,1472
12316,九州,鹿児島県,Books Misumi 鹿屋店,鹿屋市寿7-12-1,Misumi,ＭＩＳＵＭＩ,2021-06-23,2022-02-24,1774
12317,九州,福岡県,（仮称）福岡市青果市場跡地活用事業,福岡市博多区那珂六丁目351番地１,博多那珂６開発特定目的会社,未定,2021-05-14,2022-04-01,46380
12318,九州,熊本県,ロッキー水前寺店,熊本市中央区水前寺2丁目4-15,ロッキー,ロッキー,2021-05-06,2022-01-04,1670


In [ ]:
# 今度は欠損値ゼロになっている
df.isnull().sum()

局名          0
都道府県        0
大規模小売店舗名    0
所在地         0
建物設置者名      0
小売業者名       0
届出日         0
開店予定日       0
店舗面積        0
dtype: int64

## データクレンジングはここで終了